# Feature Selection using BPSO for SDP

installations

import libraries 

In [1]:
import numpy as np
import seaborn as sns
import pandas as pd
import os
import pyswarms as ps

%load_ext autoreload
%autoreload 2
%matplotlib inline

### The methods 

Dataset prepration

In [2]:
def data_prepration(file):
    df=pd.read_csv(file)
    if( 'version' in file):
        X=np.array(df.drop(['bug','name.1','name', 'version'], axis=1))
    else:
        X=np.array(df.drop('bug', axis=1))
    y=np.array(df.bug)
    
    return X,y

plot data per feature (each class with different color)

In [3]:
def plot_features(features,labels):
    df = pd.DataFrame(features)
    df['labels'] = pd.Series(labels)

    sns.pairplot(df, hue='labels' , diag_kind='hist')

fitness function for each particle

In [4]:
def f_per_particle(m):
    if(clas=='NB'):
        classifier = GaussianNB()
    elif(clas=='SVM linear'):
        classifier = svm.SVC(kernel='linear')
    elif(clas=='KNN'):
        classifier = KNeighborsClassifier(n_neighbors=5)
    elif(clas=='SVM RBF'):
        classifier = svm.SVC(kernel='rbf')
    """Computes for the fitness function per particle

    Inputs
    ------
    m : numpy.ndarray
        Binary mask that can be obtained from BinaryPSO, will
        be used to mask features.

    Returns
    -------
    numpy.ndarray
        Computed finess function
    """
    
    # Get the subset of the features from the binary mask
    count_non_zero=np.count_nonzero(m)
    if np.count_nonzero(m) == 0:
        X_subset = X_train
        X_subset_test= X_test
        count_non_zero=np.count_nonzero(m)
    else:
        X_subset = X_train[:,m==1]
        X_subset_test= X_test[:,m==1]
        
    
    # Perform classification and compute the fitness function
    classifier.fit(X_subset, y_train)
    j = (classifier.predict(X_subset_test) != y_test).mean() + count_non_zero
    
    
    
    
#     j=np.count_nonzero(m)

    return j

Define fitness function for whole swarm

In [5]:
def f(x):
    """Higher-level method to do classification in the
    whole swarm.

    Inputs
    ------
    x: numpy.ndarray of shape (n_particles, dimensions)
        The swarm that will perform the search

    Returns
    -------
    numpy.ndarray of shape (n_particles, )
        The computed loss for each particle
    """
    n_particles = x.shape[0]
    j = [f_per_particle(x[i]) for i in range(n_particles)]
    
    return np.array(j)

select the features from the set of features X using BPSO on the specfied parameters

In [6]:
def select_features(nparticles):
  
    total_features = X_train.shape[1]
    
    dimensions = total_features
    
    optimizer = ps.discrete.BinaryPSO(n_particles=nparticles, dimensions=dimensions, options=options)
    
    # Perform optimization
    cost, pos = optimizer.optimize(f, iters=iters)
    
    
    print("cost: "+str(cost))
    fr.write(clas+"cost= "+str(cost)+"\nNumber of selected features:"+str(len([i for i in pos if i==1]))+"\nSelected features:"+str(pos))
    return cost, pos

Train clssifier on the selected features (corrposining to pos which =1)

In [7]:
def train_and_test(clas,X_train,X_test,y_train,y_test,pos, fr):   
    
    if(clas=='NB'):
        classifier = GaussianNB()
    elif(clas=='SVM linear'):
        classifier = svm.SVC(kernel='linear')
    elif(clas=='KNN'):
        classifier = KNeighborsClassifier(n_neighbors=5)
    elif(clas=='SVM RBF'):
        classifier = svm.SVC(kernel='rbf')
        
    # Get the selected features from the final positions
    X_selected_features = X_train[:,pos==1]  # subset
    # Perform classification and store performance in P
    classifier.fit(X_selected_features,y_train)
    selected_test= X_test[:,pos==1]
    
    y_pred= classifier.predict(selected_test)    
    report=classification_report(y_test,y_pred)
    subset_performance = (y_pred== y_test).mean()

    # Compute performance
#     subset_performance = (classifier.predict(selected_test) == y_test).mean()
    fr.write("\nsubset_performance:"+str(subset_performance)+"\n")
    fr.write("\nclassification_report:"+str(report)+"\n")
    
    print('Subset performance: %.3f' % (subset_performance))

## The main cell 

here the classifier is created, options, number of iterations are set, and other methods are called

each results for each dataset are stored in results folder with file of the classifier name

In [8]:
    import time
    from sklearn.model_selection import train_test_split
    from sklearn.metrics import classification_report

#     from sklearn.naive_bayes import GaussianNB
#     classifier = GaussianNB()
#     clas='NB'

    # from sklearn import svm
    # classifier = svm.SVC(kernel='linear')
    # clas='SVM linear'

    from sklearn import svm
    classifier = svm.SVC(kernel='rbf')
    clas='SVM RBF'

#     from sklearn.neighbors import KNeighborsClassifier
#     classifier = KNeighborsClassifier(n_neighbors=5)
#     clas='KNN'

    # from sklearn import linear_model
    # classifier = linear_model.LogisticRegression(solver='lbfgs', multi_class='auto')
    # clas='Linear Regression'

    options = {'c1': 0.6, 'c2': 0.3, 'w': 0.4, 'k':20, 'p':2}
    # k : number of neighbors to be considered. Must be a positive integer less than n_partcles 
    # p: the Minkowski p-norm to use. 1 is the sum-of-absolute values (or L1 distance) while 2 is the Euclidean (or L2) distance
    iters= 100

    start= time.time()
    
    
    fr = open("Test/BPSO merged DS1/"+clas+".txt", "a")
    fr.write('parameters: '+str(options)+"\n")
    
    df=pd.read_csv("merged selected DS training .csv")
    X_train= np.array(df.drop('bug', axis=1))
    y_train= np.array(df.bug)
    df=pd.read_csv("merged selected DS testing .csv")
    X_test=np.array(df.drop('bug', axis=1))
    y_test= np.array(df.bug)
    
    
    cost, pos = select_features(20)
    train_and_test(clas,X_train,X_test,y_train,y_test,pos,fr)
    period= time.time() - start 
    fr.write('number of particle {}:\n'.format(20))
    fr.write("best pos cost= "+str(cost)+"\nNumber of selected features:"+str(len([i for i in pos if i==1]))+"\nSelected features:"+str(pos))
    
    fr.write('\nFS, training, and testing time= '+str(period)+" seconds\n")
    fr.close()

2021-12-03 17:45:40,041 - pyswarms.discrete.binary - INFO - Optimize for 100 iters with {'c1': 0.6, 'c2': 0.3, 'w': 0.4, 'k': 20, 'p': 2}
pyswarms.discrete.binary:   0%|                                                                                  |0/100c:\users\afnan\miniconda3\envs\tf\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
c:\users\afnan\miniconda3\envs\tf\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
c:\users\afnan\miniconda3\envs\tf\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value o

c:\users\afnan\miniconda3\envs\tf\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
c:\users\afnan\miniconda3\envs\tf\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
c:\users\afnan\miniconda3\envs\tf\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
c:\users\afnan\miniconda3\envs\tf\lib\site-packages\

c:\users\afnan\miniconda3\envs\tf\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
c:\users\afnan\miniconda3\envs\tf\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
c:\users\afnan\miniconda3\envs\tf\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
c:\users\afnan\miniconda3\envs\tf\lib\site-packages\

c:\users\afnan\miniconda3\envs\tf\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
c:\users\afnan\miniconda3\envs\tf\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
c:\users\afnan\miniconda3\envs\tf\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
pyswarms.discrete.binary:   4%|██▋                  

c:\users\afnan\miniconda3\envs\tf\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
c:\users\afnan\miniconda3\envs\tf\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
c:\users\afnan\miniconda3\envs\tf\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
c:\users\afnan\miniconda3\envs\tf\lib\site-packages\

...

c:\users\afnan\miniconda3\envs\tf\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
c:\users\afnan\miniconda3\envs\tf\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
c:\users\afnan\miniconda3\envs\tf\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
c:\users\afnan\miniconda3\envs\tf\lib\site-packages\

c:\users\afnan\miniconda3\envs\tf\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
c:\users\afnan\miniconda3\envs\tf\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
c:\users\afnan\miniconda3\envs\tf\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
c:\users\afnan\miniconda3\envs\tf\lib\site-packages\

c:\users\afnan\miniconda3\envs\tf\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
c:\users\afnan\miniconda3\envs\tf\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
c:\users\afnan\miniconda3\envs\tf\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
c:\users\afnan\miniconda3\envs\tf\lib\site-packages\

c:\users\afnan\miniconda3\envs\tf\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


cost: 4.363636363636363
Subset performance: 0.636
